In [12]:
import lec_xml2 as lec
import IPython.display as display 
import func_mcep
import functions
import soundfile
import numpy as np

In [3]:

import sys, os
import xml.etree.ElementTree as ET # xmlを読むときいろいろ助けてくれるやつ
import re
import numpy as np


note_dict = {'C':0,'D':2,'E':4,'F':5,'G':7,'A':9,'B':11}

key_ids = [60, 67, 62, 69, 64, 71, 66, 61, 68, 63, 70, 65]
#          C4, G4, D4, A4, E4, B4,F#4,C#4,G#4,D#4,A#4, F4


def readxml_py(fname):
    tree = ET.parse(fname) # xmlを木構造にしてくれているらしい
    score = tree.getroot() # 根(一番外側)

    notenum = 0
    dur_total = 0
    array_length = 0
    keyfif = 0
    tempo = 120.0
    
    singtime = np.array([])
    lyric = np.array([])
    pitch = np.array([])

    for measure in score.iter('measure'): # 小節ごとに処理する
        #print("------小節線------")

        attributes = measure.find('attributes')
        # 楽譜を記述する上での情報を取得したい
        if attributes is not None: # 小節内にattributesという項目があれば...という意味合い
            time = attributes.find('time')
            divi = attributes.find('divisions')
            key = attributes.find('key')

            if time is not None:
                beats = int(time.find('beats').text) # 拍子
                beat_type = int(time.find('beat-type').text) # 何分の
            if divi is not None:
                divisions = int(divi.text) # 一拍を何個に分割するか(音符の長さの分解能・最小単位)
            if key is not None:
                keyfif = int(key.find('fifths').text) # 調

            dur_msr = beats * divisions * 4 / beat_type # 1小節に最小単位が何個入るか

        direction = measure.find('direction')
        if direction is not None:
            t = direction.find('sound')
            tempo = float(t.attrib['tempo'])
            
        #print('tempo:' + str(tempo))

        for note in measure.iter('note'):
            dur = int(note.find('duration').text) # 音符/休符の長さ(最小単位何個分か)
            singtime = np.append(singtime,dur / (tempo / 60 * divisions))
            
            rest = note.find('rest')
            if rest is not None:
                # 休符のときの処理
                lyric = np.append(lyric,"sil")
                pitch = np.append(pitch,0)

            else:
                tie_flg = 0
                tie_start = 0
                tie_end = 0

                # タイに関する処理の例
                # tie_flg 0:タイなし
                # tie_flg 1:タイ開始
                # tie_flg 2:タイ途中
                # tie_flg 3:タイ終了
                for tie in note.iter('tie'):
                    if re.match('start',tie.attrib['type']):
                        tie_start = 1
                        tie_flg = 1
                    if re.match('stop',tie.attrib['type']):
                        tie_end = 1
                        tie_flg = 3
                if tie_start * tie_end == 1:
                    tie_flg = 2

                if tie_flg < 2:
                    lyr = note.find('lyric')
                    if lyr is not None:
                        # 歌詞があるときの処理
                        lyric = np.append(lyric,lyr.find('text').text)
                        pass
                    
                    # 音の高さに関する処理
                    note_name = note.find('pitch').find('step').text
                    note_alter = note.find('pitch').find('alter')
                    note_oct = note.find('pitch').find('octave').text

                    note_pitch = 12*(int(note_oct)+1) + note_dict[note_name]
                    if note_alter is not None:
                        note_pitch += int(note_alter.text)
                        
                    pitch = np.append(pitch,note_pitch)

                if tie_flg == 0 or tie_flg == 3:
                    # 音符が終わるときにする処理
                    pass

    return singtime,lyric,pitch

In [20]:
def songmake(file):
    
    singtime,lyric,pitch = readxml_py(file)
    
    syllable = lyric[0]
    time = singtime[0]
    midi = pitch[0]

    directory = './RITSU/Normal/'
    song,fs = soundfile.read(directory + syllable + '.wav')

    f0,mcep,bap = functions.extract(song,fs)
    f0,mcep,bap = func_mcep.leng(song,f0,mcep,bap,time,fs)
    f0 = func_mcep.makePitch(f0,midi)

    for tone in range(len(lyric) - 1):
    
        syllable = lyric[tone + 1]
        time = singtime[tone + 1]
        midi = pitch[tone + 1]
    
        addsong,fs = soundfile.read(directory + syllable + '.wav')
    
        addf0,addmcep,addbap = functions.extract(addsong,fs)
        addf0,addmcep,addbap = func_mcep.leng(addsong,addf0,addmcep,addbap,time,fs)
        addf0 = func_mcep.makePitch(addf0,midi)
    
        f0,mcep,bap = func_mcep.connect(f0,addf0,mcep,addmcep,bap,addbap)
    
    newsong = functions.audiomake(f0,mcep,bap,fs)

    return newsong

In [21]:
file = 'げんこつ.xml'
song = songmake(file)
display.Audio(song,rate=fs)